# Essential Scientific Libraries

โดย ดร. อัญชลิสา แต้ตระกูล <br>
ภาควิชาวิศวกรรมคอมพิวเตอร์ คณะวิศวกรรมศาสตร์ มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี <br>
E-mail: unchalisa.tae@mail.kmutt.ac.th

TA #1: ธนกฤต คล้ายแก้ว (เอ้)<br>
TA #2: ธนกฤต ผังวิวัฒน์ (พั้นช์)

# Classification Lab

### Part 1: Load  data

Import "bank-data.csv"

In [ ]:
import pandas as pd
bankData = pd.read_csv('bank-data.csv', sep = ';')
bankData.head()

### Part 2: Preprocess data

Preprocess the dataset as you have done before

In [ ]:
bankData.shape

#### 2.1 Binary encoding

Use LabelEncoder to encode the following columns:
- y
- default
- housing
- loan

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#example
bankData['y'] = le.fit_transform(bankData['y'])
bankData.head()

In [ ]:
#Encode the remaining columns




bankData.head()

#### 2.2 Convert categorical variables into dummy columns

(1) Use pd.get_dummies to convert the following categorical variales into dummy columns
- job
- maritial
- education
- contact
- month
- poutcome

(2) Drop columns that have been converted

In [ ]:
#example
bankData = pd.concat([bankData,pd.get_dummies(bankData['job'],prefix='job')],axis=1)
bankData = bankData.drop(columns=['job'])
bankData.head()

In [ ]:
#convert the remaining categorical variables




bankData = bankData.drop(columns=['marital', 'education', 'contact', 'month', 'poutcome'])

In [ ]:
bankData.info()

#### 2.3 Train/Test separation

Perform hold-out method
- 60% training set
- 40% testing set

In [ ]:
bankData_train = bankData.sample(frac = 0.6)
bankData_test = bankData.drop(bankData_train.index)
print(pd.crosstab(bankData_train['y'],columns = 'count'))
print(pd.crosstab(bankData_test['y'],columns = 'count'))

##### X/y separation

In [ ]:
bankData_train_y = bankData_train['y']
bankData_train_X = bankData_train.copy()
del bankData_train_X['y']

bankData_test_y = bankData_test['y']
bankData_test_X = bankData_test.copy()
del bankData_test_X['y']

## Decision Tree

### Part 3: Train a model

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_leaf=30, max_depth=5)
clf = clf.fit(bankData_train_X, bankData_train_y)
print(clf)

##### Tree Visualization

You MUST first install 'graphviz' in order to run the following code.

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None, 
                              feature_names=bankData_train_X.columns,
                              class_names=['0','1'],
                              filled=True, rounded=True,
                              special_characters=True, rotate=True)
graph = graphviz.Source(dot_data)
graph.render('dtree_render')

##### Variable importance

In [ ]:
tree_feature = pd.DataFrame({'feature':bankData_train_X.columns,
                             'Score':clf.feature_importances_})

tree_feature.sort_values(by = 'Score', ascending=False)

##### Prediction

In [ ]:
clf.predict(bankData_test_X)

In [ ]:
clf.predict_proba(bankData_test_X)

### Part 4: Model Evaluation

Evaluation metrics
- confusion metrix
- accuracy
- precision, recall, f1-score

In [ ]:
#confusion metrix
res = clf.predict(bankData_test_X)
pd.crosstab(bankData_test_y, res)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print("Accuracy:\t %.3f" %accuracy_score(bankData_test_y, res))
print(classification_report(bankData_test_y, res))

### Part 5: Model tuning

#### Note:

After building the decision tree classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

##### Pruning Parameters
- max_leaf_nodes
    - Reduce the number of leaf nodes
- min_samples_leaf
    - Restrict the size of sample leaf
    - Minimum sample size in terminal nodes can be fixed to 30, 100, 300 or 5% of total
- max_depth
    - Reduce the depth of the tree to build a generalized tree
    - Set the depth of the tree to 3, 5, 10 depending after verification on test data

## Random Forest

### Part 3: Train a model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_random = RandomForestClassifier(random_state=0)
clf_random = clf_random.fit(bankData_train_X, bankData_train_y)

print(clf_random)

##### Variable importance

### Part 4: Model Evaluation

##### Prediction

Evaluation metrics
- confusion metrix
- accuracy
- precision, recall, f1-score

### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

##### Tuning Parameters
- max_depth
- max_features
- max_leaf_nodes

## kNN

### Part 3: Data processing

#### 1. Distances

In [ ]:
# Euclidean distance is suitable for most of numeric data
euclidean_distances(bankData[0:3])

In [ ]:
# Manhattan distance is less sensitive to outlier
from sklearn.metrics.pairwise import manhattan_distances
manhattan_distances(bankData[0:3])

In [ ]:
# Distance with scaling data
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

bankData_scaled = pd.DataFrame(
                    mms.fit_transform(bankData),
                    columns=bankData.columns)

euclidean_distances(bankData_scaled[0:3])

#### 2. Look-alike

In [ ]:
d = euclidean_distances(bankData_scaled,
                        bankData_scaled[0:1])

result = bankData
result['d'] = d
result.sort_values(by='d')

### Part 3: Train a model: kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5,
                           metric='euclidean',
                           n_jobs=-1)
clf.fit(X_train,y_train)

In [ ]:
res = clf.predict(X_test)
pd.crosstab(y_test, res)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print("Accuracy:\t %.3f" %accuracy_score(y_test, res))
print(classification_report(y_test, res))

#### Weighted voting kNN

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors=5,
                            metric='euclidean',
                            n_jobs=-1,
                            weights='distance')
clf1.fit(X_train,y_train)

In [ ]:
#test and evaluate
res1 = clf1.predict(X_test)
pd.crosstab(y_test, res1)

In [ ]:
print("Accuracy:\t %.3f" %accuracy_score(y_test, res1))
print(classification_report(y_test, res1))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f820e82e-2881-4678-9c4d-8fe7d1290c83' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>